In [10]:
import duckdb as ddb
import pandas as pd

con = ddb.connect("../air_quality.db")

In [11]:
df = con.query("select * from raw.air_quality where parameter in ('so2', 'pm10', 'pm25')").to_df()
df.head()

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime
0,288,3725,Hawthorne-288,2024-01-01 08:00:00,40.733501,-111.871696,so2,ppm,0.0017,01,2024,2024-12-18 14:17:59.479
1,288,3725,Hawthorne-288,2024-01-01 09:00:00,40.733501,-111.871696,so2,ppm,0.0018,01,2024,2024-12-18 14:17:59.479
2,288,3725,Hawthorne-288,2024-01-01 10:00:00,40.733501,-111.871696,so2,ppm,0.0016,01,2024,2024-12-18 14:17:59.479
3,288,3725,Hawthorne-288,2024-01-01 11:00:00,40.733501,-111.871696,so2,ppm,0.0016,01,2024,2024-12-18 14:17:59.479
4,288,3725,Hawthorne-288,2024-01-01 12:00:00,40.733501,-111.871696,so2,ppm,0.0018,01,2024,2024-12-18 14:17:59.479


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44955 entries, 0 to 44954
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   location_id         44955 non-null  int64         
 1   sensors_id          44955 non-null  int64         
 2   location            44955 non-null  object        
 3   datetime            44955 non-null  datetime64[us]
 4   lat                 44955 non-null  float64       
 5   lon                 44955 non-null  float64       
 6   parameter           44955 non-null  object        
 7   units               44955 non-null  object        
 8   value               44955 non-null  float64       
 9   month               44955 non-null  object        
 10  year                44955 non-null  int64         
 11  ingestion_datetime  44955 non-null  datetime64[us]
dtypes: datetime64[us](2), float64(3), int64(3), object(4)
memory usage: 4.1+ MB


In [13]:
df.describe()

,location_id,sensors_id,datetime,lat,lon,value,year,ingestion_datetime
count,4.495500e+04,4.495500e+04,44955,44955.000000,44955.000000,44955.000000,44955.0,44955
mean,2.606131e+05,1.445301e+06,2024-02-15 15:00:12.412412,40.646904,-111.909607,6.332745,2024.0,2024-12-18 14:18:40.701777
min,2.880000e+02,5.590000e+02,2024-01-01 08:00:00,40.341400,-112.355000,-4.000000,2024.0,2024-12-18 14:17:59.479000
25%,2.168000e+03,3.897000e+03,2024-01-22 23:00:00,40.496408,-111.945000,0.113839,2024.0,2024-12-18 14:18:15.740000
50%,8.758000e+03,2.553600e+04,2024-02-15 13:00:00,40.733501,-111.894167,2.700000,2024.0,2024-12-18 14:18:42.366000
75%,2.283040e+05,1.316196e+06,2024-03-10 00:00:00,40.777100,-111.849583,7.600000,2024.0,2024-12-18 14:19:01.984000
max,2.027500e+06,7.978757e+06,2024-04-01 06:00:00,40.897999,-111.711260,1343.000000,2024.0,2024-12-18 14:19:37.190000
std,5.559777e+05,2.451546e+06,NaN,0.164403,0.141996,15.723063,0.0,NaN


In [14]:
df.describe(include='O')

,location,parameter,units,month
count,44955,44955,44955,44955
unique,18,3,2,3
top,Hawthorne-288,pm25,µg/m³,01
freq,6221,27777,36551,15786


In [15]:
df[df.duplicated(subset=["location_id", "parameter", "units", "value", "datetime"])]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime
40219,1236049,7970867,900S Sunnyside-2490680,2024-03-09 18:00:00,40.749303,-111.849583,pm10,µg/m³,7.150974,03,2024,2024-12-18 14:19:22.837
40220,1236049,7970867,900S Sunnyside-2490680,2024-03-09 19:00:00,40.749303,-111.849583,pm10,µg/m³,6.747727,03,2024,2024-12-18 14:19:22.837
42859,2026421,7978101,Constellation Labs Pleasant Grove-1996396,2024-03-09 18:00:00,40.357937,-111.711260,pm10,µg/m³,8.706845,03,2024,2024-12-18 14:19:32.896
42860,2026421,7978101,Constellation Labs Pleasant Grove-1996396,2024-03-09 19:00:00,40.357937,-111.711260,pm10,µg/m³,6.348214,03,2024,2024-12-18 14:19:32.896


In [16]:
df.groupby(by="parameter", as_index=False).count()

,parameter,location_id,sensors_id,location,datetime,lat,lon,units,value,month,year,ingestion_datetime
0,pm10,8774,8774,8774,8774,8774,8774,8774,8774,8774,8774,8774
1,pm25,27777,27777,27777,27777,27777,27777,27777,27777,27777,27777,27777
2,so2,8404,8404,8404,8404,8404,8404,8404,8404,8404,8404,8404


In [17]:
df[df["value"] < 0]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime
395,288,559,Hawthorne-288,2024-01-06 07:00:00,40.733501,-111.871696,pm10,µg/m³,-2.0,01,2024,2024-12-18 14:17:59.479
400,288,559,Hawthorne-288,2024-01-07 12:00:00,40.733501,-111.871696,pm10,µg/m³,-3.0,01,2024,2024-12-18 14:17:59.479
460,288,559,Hawthorne-288,2024-01-07 06:00:00,40.733501,-111.871696,pm10,µg/m³,-1.0,01,2024,2024-12-18 14:17:59.479
461,288,559,Hawthorne-288,2024-01-07 07:00:00,40.733501,-111.871696,pm10,µg/m³,-3.0,01,2024,2024-12-18 14:17:59.479
467,288,559,Hawthorne-288,2024-01-13 13:00:00,40.733501,-111.871696,pm10,µg/m³,-1.0,01,2024,2024-12-18 14:17:59.479
...,...,...,...,...,...,...,...,...,...,...,...,...
36872,228304,1562372,Utah Tech. Center-240600,2024-03-31 07:00:00,40.777100,-111.945000,pm10,µg/m³,-1.0,03,2024,2024-12-18 14:19:04.088
36877,228304,1562372,Utah Tech. Center-240600,2024-03-31 12:00:00,40.777100,-111.945000,pm10,µg/m³,-4.0,03,2024,2024-12-18 14:19:04.088
36880,228304,1562372,Utah Tech. Center-240600,2024-03-31 15:00:00,40.777100,-111.945000,pm10,µg/m³,-3.0,03,2024,2024-12-18 14:19:04.088
36890,228304,1562372,Utah Tech. Center-240600,2024-04-01 01:00:00,40.777100,-111.945000,pm10,µg/m³,-2.0,03,2024,2024-12-18 14:19:04.088


In [18]:
con.close()